## Prompts for locations

In [1]:
location_prompts = [
    "steampunk city with skyscrapers",
    # "cyberpunk village in Japanese rustic style",
    # "fantasy dungsseons and dragons",
    # "noir city from 1930s",
    # "StarTrek inspired spaceship",
    # "undeground mine of goblins",
    # "SuperMario style magic land plain",
    # "SuperMario style magic land beach",
]

## Generating locations 

In [2]:
from holodeck import initialize_location
from holodeck import generate_location_and_encounters
import os
from tqdm.notebook import tqdm
import traceback

import concurrent.futures
from tqdm import tqdm

def generate_location(prompt):
    location_dict, encounters_list = generate_location_and_encounters(prompt)
    if location_dict:
        try:
            location = initialize_location(location_dict, encounters_list)
            return location
        except Exception as e:
            print("Error: ", e)
            traceback.print_exc()
    else:
        print(f"GENERATING FROM '{prompt}' failed!")
        return None

locations = []
with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    results = list(tqdm(executor.map(generate_location, location_prompts), total=len(location_prompts), desc="Generating locations"))
    locations += [r for r in results if r is not None]


assert len(locations) == len(location_prompts), "some locations are failed to generate"

locations

Generating locations:   0%|          | 0/1 [00:00<?, ?it/s]

<
{'name': 'Steampunk City of X', 'description': 'A bustling steampunk city, with towering metal skyscrapers reaching as far as the eye can see. The streets are filled with people going about their daily lives and the roads are filled with hovercars and carriages. In the night the city is lit up with an array of neon lights.\n', 'buildings': [{'name': 'Clock Tower', 'description': 'A tall, stately clock tower in the center of the city. It strikes a bell every hour, keeping the citizens in tune with the time.', 'enterable': True}, {'name': 'Airship Hangar', 'description': 'A large metal building, filled with a variety of advanced airships of every kind. All sizes, shapes and designs can be found here.', 'enterable': True}], 'ways': [{'name': 'Main Street', 'description': 'The main thoroughfare of the city, lined with tall buildings and filled with a constant stream of hovercars and carriages.'}, {'name': 'Upper Avenue', 'description': 'A wide avenue on the north side of the city, overlo

Traceback (most recent call last):
  File "/Users/standard/SRC/Holodeck/.venv/lib/python3.10/site-packages/sqlalchemy/orm/mapper.py", line 2036, in get_property
    return self._props[key]
KeyError: 'location'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/var/folders/gf/k97vplfs0kq5rgx4cp5557sh0000gn/T/ipykernel_14789/1867708835.py", line 14, in generate_location
    location = initialize_location(location_dict, encounters_list)
  File "/Users/standard/SRC/Holodeck/holodeck/gpt_game_gen.py", line 40, in initialize_location
    location = Location(location_dict.get('name', f"Location {get_location_number()}"), location_dict.get('description', ""))
  File "<string>", line 4, in __init__
  File "/Users/standard/SRC/Holodeck/.venv/lib/python3.10/site-packages/sqlalchemy/orm/state.py", line 476, in _initialize_instance
    manager.dispatch.init(self, args, kwargs)
  File "/Users/standard/SRC/Holodeck/.venv/lib/python3.10/s

<
[{'probability': 0.1, 'description': 'As you walk down Main Street, you find a damaged hovercar', 'actions': [{'type': 'item', 'description': 'Damaged hovercar, still in working condition'}], 'trigger': {'type': 'way', 'way': 'Main Street'}}, {'probability': 0.3, 'description': 'On the Clock Tower, you hear a strange ticking sound coming from inside', 'actions': [{'type': 'building', 'name': 'Mysterious Clock', 'description': 'A strange ticking sound coming from inside the Clock Tower'}], 'trigger': {'type': 'building', 'building': 'Clock Tower'}}, {'probability': 0.02, 'description': 'As you approach the Mysterious Clock, a giant robot appears!', 'actions': [{'type': 'character', 'name': 'Mysterious Robot', 'description': 'A giant robot, made of a strange mix of materials'}], 'trigger': {'type': 'building', 'building': 'Mysterious Clock'}}]

Error:  Mapper 'mapped class Building->building' has no property 'location'


AssertionError: some locations are failed to generate

## Generate Image Prompts

In [ ]:
from holodeck.gpt_text import \
        generate_object_image_prompt, \
        generate_building_image_prompt, \
        generate_location_image_prompt

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

locations_prompts = []
objects_image_prompts = []
buildings_image_prompts = []

def generate_location_images(location):
    return (location, generate_location_image_prompt(location))

def generate_object_images(location):
    prompts = []
    for o in location.objects:
        prompts.append((o, generate_object_image_prompt(o, location)))
    return prompts

def generate_building_images(location):
    prompts = []
    for b in location.all_buildings:
        prompts.append((b, generate_building_image_prompt(b, location)))
    return prompts

with ThreadPoolExecutor() as executor:
    location_image_futures = list(tqdm(executor.map(generate_location_images, locations), desc="Locations"))
    object_image_futures = [executor.submit(generate_object_images, location) for location in locations]
    building_image_futures = [executor.submit(generate_building_images, location) for location in locations]

    for location, prompt in location_image_futures:
        locations_prompts.append((location, prompt))
    for f in object_image_futures:
        for prompt in f.result():
            objects_image_prompts.append(prompt)
    for f in building_image_futures:
        for prompt in f.result():
            buildings_image_prompts.append(prompt)


img_prompts = locations_prompts + objects_image_prompts + buildings_image_prompts

obj_number = 0
for location in locations:
    obj_number += 1
    obj_number += len(location.all_buildings)
    obj_number += len(location.objects)


assert len(img_prompts) == obj_number, "not all descriptions got generated!"



## Generate Images

In [ ]:
from holodeck.gpt_image import generate_image
from IPython.display import display, Markdown
import PIL.Image as Image
import random


random.shuffle(img_prompts)

# img_prompts = img_prompts[:9]

images = []

for obj, gen_options in tqdm(img_prompts, desc="Images"):
    prompt = gen_options['prompt']
    negative_prompt = gen_options['negative_prompt']
    image_bytes = await generate_image(prompt=prompt, negative_prompt=negative_prompt)
    image = Image.open(image_bytes)
    display(Markdown(f"### {obj.name}"))
    display(Markdown(f"""
- {prompt}
  - *negative: {negative_prompt}*
    """))
    display(image)
    image_file_name = f".images/{obj.name}.png"
    image.save(image_file_name)
    image.close()
    image_bytes.close()
    del image_bytes
    images.append((obj, prompt, image_file_name))

len(images)


## Display Images

In [ ]:
import textwrap
import matplotlib.pyplot as plt
from matplotlib.patheffects import withStroke
from PIL.Image import Resampling

text_width = 30
dpi = 600.0
fig_width = 3.234

columns = 4
pic_width = fig_width * 0.27


def plot_images(images):

    # Estimate the number of rows needed based on the number of images
    num_rows = (len(images) + columns - 1) // columns
    fig_height = 0
    # Get the height of the first image adjusted for scale
    with Image.open(images[0][2]) as img:
        w, h = img.size
        aspect_ratio = float(w) / h
        pic_height = pic_width / aspect_ratio

        fig_height = pic_height * num_rows * 1.6

    fig = plt.figure(figsize=(fig_width, fig_height), dpi=dpi)

    # Define the path effect for the outline
    outline_effect = withStroke(linewidth=0.3, foreground='black')

    # Loop over the images and create a subplot for each
    for i, (obj, prompt, image_file_name) in enumerate(images):
        with Image.open(image_file_name) as image:
        
            # Resize the image
            w, h = image.size
            aspect_ratio = float(w) / h
            new_width = int(pic_width * dpi)
            new_height = int(new_width / aspect_ratio)
            with image.resize((new_width, new_height), Resampling.LANCZOS) as image_resized:
                # Create a subplot for the image
                ax = fig.add_subplot(len(images) // columns + 1, columns, i + 1)
                
                # Display the image
                ax.imshow(image_resized)

        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect('equal') # set aspect ratio to 1:1
        
        # Set the title to the obj.name
        # ax.set_title(obj.name, fontsize=4, color='magenta', pad=-10)
        
        # Wrap the prompt text to the desired width
        wrapped_prompt = textwrap.fill(prompt, width=text_width)
        wrapped_title = textwrap.fill(obj.name, width=14)
        
        # Display the wrapped prompt text below the title with black outline
        ax.text(0.08, -0.38, wrapped_prompt, ha='left', va='bottom', transform=ax.transAxes, fontsize=1.7, family='monospace', color='white', path_effects=[outline_effect])
        
        ax.text(1.0-0.08, -0.4, wrapped_title, ha='right', va='top', transform=ax.transAxes, fontsize=2.4, family='monospace', color='magenta', path_effects=[outline_effect])

    plt.subplots_adjust(wspace=0.03)

    fig.patch.set_facecolor('none')


plot_images(images)


In [ ]:
print("hi") 

In [ ]:
print